# Calculate

In [2]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2


from IPython.core import debugger
ist = debugger.set_trace

In [3]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [4]:
import sys
sys.path.insert(0,'../')

## CALCS

In [6]:
%%run_and_write ../typyPRISM/calculate/prism/pair_correlation.py
from typyPRISM.core.Space import Space

def pair_correlation(PRISM):
    '''Calculate the pair correlation function from a PRISM object
    
    After convergence, the stored total correlation function can 
    simply be shifted to obtain the pair-correlation function 
    i.e. the radial distribution function. 
    
    .. math::
        g(r) = h(r) + 1.0
        
    Parameters
    ----------
    PRISM: typyPRISM.core.PRISM.PRISM
        A **solved** PRISM object.
    
    
    Returns
    -------
    pairCorr: typyPRISM.core.MatrixArray.MatrixArray
        The full MatrixArray of pair correlation functions.
    
    '''
    
    if PRISM.totalCorr.space == Space.Fourier:
        PRISM.domain.MatrixArray_to_real(PRISM.totalCorr)
    
    PRISM.pairCorr = PRISM.totalCorr + 1.0
    
    return PRISM.pairCorr

Overwriting ../typyPRISM/calculate/prism/pair_correlation.py


In [7]:
%%run_and_write ../typyPRISM/calculate/prism/structure_factor.py
from typyPRISM.core.Space import Space

def structure_factor(PRISM):
    '''Calculate the structure factor from a PRISM object
    
    .. math::
        S(k) = \omega(k) + h(k)
        
    Parameters
    ----------
    PRISM: typyPRISM.core.PRISM.PRISM
        A **solved** PRISM object.
    
    Returns
    -------
    structureFactor: typyPRISM.core.MatrixArray.MatrixArray
        The full MatrixArray of structure factors
    
    '''
    
    if PRISM.totalCorr.space == Space.Real:
        PRISM.domain.MatrixArray_to_real(PRISM.totalCorr)
        
    if PRISM.intraMolCorr.space == Space.Real:
        PRISM.domain.MatrixArray_to_real(PRISM.intraMolCorr)
    
    structureFactor = PRISM.totalCorr*PRISM.pairDensityMatrix + PRISM.intraMolCorr
    
    return structureFactor

Overwriting ../typyPRISM/calculate/prism/structure_factor.py


## TESTING

In [12]:
%%run_and_write ../typyPRISM/test/CalcPRISM_TestCase.py
import unittest
import numpy as np
import typyPRISM

class CalcPRISM_TestCase(unittest.TestCase):
    def setup(self):
        '''Construct a simple but fully specified PRISM problem'''
        
        sys = typyPRISM.System(['A','B'])
        
        sys.domain = typyPRISM.Domain(dr=0.1,length=512)
        
        sys.density['A'] = 0.5
        sys.density['B'] = 0.5
        
        sys.closure.setUnset(typyPRISM.closure.PercusYevick())
        
        sys.potential.setUnset(typyPRISM.potential.HardSphere(sigma=1.0))
        
        sys.intraMolCorr['A','A'] = typyPRISM.intraMolCorr.SingleSite()
        sys.intraMolCorr['A','B'] = typyPRISM.intraMolCorr.NoIntra()
        sys.intraMolCorr['B','B'] = typyPRISM.intraMolCorr.Gaussian(sigma=1.0,length=10)
        
        PRISM = sys.createPRISM()
        
        return PRISM

    def test_pair_correlation(self):
        '''Can we calculate pair_correlations?'''
        PRISM = self.setup()
        PRISM.solve(disp=False)
        result = typyPRISM.calculate.prism.pair_correlation(PRISM)
        
    def test_structure_factor(self):
        '''Can we calculate structure factors?'''
        PRISM = self.setup()
        PRISM.solve(disp=False)
        result = typyPRISM.calculate.prism.structure_factor(PRISM)
        
        


Overwriting ../typyPRISM/test/CalcPRISM_TestCase.py


## RUN ALL

In [13]:
import unittest
suite = []
suite.append(unittest.TestLoader().loadTestsFromTestCase(CalcPRISM_TestCase))
suite = unittest.TestSuite(suite)
unittest.TextTestRunner(verbosity=2).run(suite)

test_pair_correlation (__main__.CalcPRISM_TestCase)
Can we calculate pair_correlations? ... ok
test_structure_factor (__main__.CalcPRISM_TestCase)
Can we calculate structure factors? ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.240s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>